In [1]:
import Pkg
Pkg.activate(".")

# not for the faint of heart!
# Pkg.update()

pkgs = [
"ArgParse",
"Base64",
"BioSequences",
"DataFrames",
"Dates",
"DelimitedFiles",
"FASTX",
"GLM",
"HTTP",
"JSON",
"Graphs",
"MetaGraphs",
"MD5",
"Statistics",
"StatsPlots",
"uCSV",
"CodecZlib",
"YAML",
"Revise",
"Kmers",
"StatsBase",
"ProgressMeter"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

  Activating environment at `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Project.toml`
    Updating registry at `/opt/julia/registries/General`
   Resolving package versions...
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Project.toml`
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Manifest.toml`


In [2]:
data_dir = joinpath(dirname(pwd()), "data")

"/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data"

In [3]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> "trim_galore" in readdir(x), SRR_paths)

333-element Vector{String}:
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399460"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399461"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399462"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399463"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399464"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399465"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399466"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399467"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399468"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399469"
 "/home/jovyan/work/Mycelia/projects/viral-pange

In [4]:
n_cores = max(Int(round(Sys.CPU_THREADS / 2)), 1)

24

In [ ]:
ProgressMeter.@showprogress for SRR_path in SRR_paths
    SRR = basename(SRR_path)

    out_dir = joinpath(SRR_path, "megahit")

    trimmed_forward_reads = joinpath(SRR_path, "trim_galore", "$(SRR)_1_val_1.fq.gz")
    trimmed_reverse_reads = joinpath(SRR_path, "trim_galore", "$(SRR)_2_val_2.fq.gz")

    # megahit: MEGAHIT v1.2.9

    # contact: Dinghua Li <voutcn@gmail.com>

    # Usage:
    #   megahit [options] {-1 <pe1> -2 <pe2> | --12 <pe12> | -r <se>} [-o <out_dir>]

    #   Input options that can be specified for multiple times (supporting plain text and gz/bz2 extensions)
    #     -1                       <pe1>          comma-separated list of fasta/q paired-end #1 files, paired with files in <pe2>
    #     -2                       <pe2>          comma-separated list of fasta/q paired-end #2 files, paired with files in <pe1>
    #     --12                     <pe12>         comma-separated list of interleaved fasta/q paired-end files
    #     -r/--read                <se>           comma-separated list of fasta/q single-end files

    # Optional Arguments:
    #   Basic assembly options:
    #     --min-count              <int>          minimum multiplicity for filtering (k_min+1)-mers [2]
    #     --k-list                 <int,int,..>   comma-separated list of kmer size
    #                                             all must be odd, in the range 15-255, increment <= 28)
    #                                             [21,29,39,59,79,99,119,141]

    #   Another way to set --k-list (overrides --k-list if one of them set):
    #     --k-min                  <int>          minimum kmer size (<= 255), must be odd number [21]
    #     --k-max                  <int>          maximum kmer size (<= 255), must be odd number [141]
    #     --k-step                 <int>          increment of kmer size of each iteration (<= 28), must be even number [12]

    #   Advanced assembly options:
    #     --no-mercy                              do not add mercy kmers
    #     --bubble-level           <int>          intensity of bubble merging (0-2), 0 to disable [2]
    #     --merge-level            <l,s>          merge complex bubbles of length <= l*kmer_size and similarity >= s [20,0.95]
    #     --prune-level            <int>          strength of low depth pruning (0-3) [2]
    #     --prune-depth            <int>          remove unitigs with avg kmer depth less than this value [2]
    #     --disconnect-ratio       <float>        disconnect unitigs if its depth is less than this ratio times 
    #                                             the total depth of itself and its siblings [0.1]  
    #     --low-local-ratio        <float>        remove unitigs if its depth is less than this ratio times
    #                                             the average depth of the neighborhoods [0.2]
    #     --max-tip-len            <int>          remove tips less than this value [2*k]
    #     --cleaning-rounds        <int>          number of rounds for graph cleanning [5]
    #     --no-local                              disable local assembly
    #     --kmin-1pass                            use 1pass mode to build SdBG of k_min

    #   Presets parameters:
    #     --presets                <str>          override a group of parameters; possible values:
    #                                             meta-sensitive: '--min-count 1 --k-list 21,29,39,49,...,129,141'
    #                                             meta-large: '--k-min 27 --k-max 127 --k-step 10'
    #                                             (large & complex metagenomes, like soil)

    #   Hardware options:
    #     -t/--num-cpu-threads     <int>          number of CPU threads [# of logical processors]

    #   Output options:
    #     -o/--out-dir             <string>       output directory [./megahit_out]
    #     --out-prefix             <string>       output prefix (the contig file will be OUT_DIR/OUT_PREFIX.contigs.fa)
    #     --min-contig-len         <int>          minimum length of contigs to output [200]

    # median_read_length = Statistics.median(vcat(Mycelia.determine_read_lengths(trimmed_forward_reads), Mycelia.determine_read_lengths(trimmed_reverse_reads)))

    # k_lengths = [i for i in [21,29,39,59,79,99,119,141] if i < joint_median_read_length]
    # k_lengths_string = join(string.(k_lengths), ',')

    # 2023-02-19 00:51:46 - ALL DONE. Time elapsed: 2031.380950 seconds 

    initial_assembled_fasta = "$(out_dir)/final.contigs.fa"
    assembled_fastg = replace(initial_assembled_fasta, ".fa" => ".fastg")

    if !isfile(initial_assembled_fasta)
        run(`megahit --num-cpu-threads $(n_cores) -1 $trimmed_forward_reads -2 $trimmed_reverse_reads -o $(out_dir)`)
        # run(`conda run --live-stream --no-capture-output -n viral-pangenome-discovery megahit --num-cpu-threads $(n_cores) -1 $trimmed_forward_reads -2 $trimmed_reverse_reads -o $(out_dir)`)
    end

    # read in the assembled fasta file and parse contig identifiers to get final k length
    final_k_lengths = unique([replace(first(split(FASTX.identifier(record), '_')), r"^k" => "") for record in Mycelia.open_fastx(initial_assembled_fasta)])
    @assert length(final_k_lengths) == 1
    final_k_length = parse(Int, first(final_k_lengths))
    if !isfile(assembled_fastg)
        run(pipeline(`megahit_toolkit contig2fastg $(final_k_length) $(initial_assembled_fasta)`, assembled_fastg))
    end

    assembled_gfa = "$(assembled_fastg).gfa"
    if !isfile(assembled_gfa)
        run(`Bandage reduce $(assembled_fastg) $(assembled_gfa)`)
    end

    assembled_fasta = assembled_gfa * ".fna"
    if !isfile(assembled_fasta)
        open(assembled_fasta, "w") do io
            fastx_io = FASTX.FASTA.Writer(io)
            gfa_graph = Mycelia.parse_gfa(assembled_gfa)
            for v in Graphs.vertices(gfa_graph)
                record = FASTX.FASTA.Record(gfa_graph.vprops[v][:identifier], gfa_graph.vprops[v][:sequence])
                write(fastx_io, record)
            end
            close(fastx_io)
        end
    end

    # generate a bandage plot of the assembly graph
    bandage_outfile = "$(assembled_gfa).bandage.jpg"
    if !isfile(bandage_outfile)
        run(`Bandage image $(assembled_gfa) $bandage_outfile`)
    end

    # map reads to the assembly and run qualimap QC
    bwt_index = "$(assembled_fasta).bwt"
    if !isfile(bwt_index)
        run(`bwa index $(assembled_fasta)`)
    end

    # mapped_reads_bam = "$(assembled_fasta).bwa.sorted.marked_duplicates.bam"
    mapped_reads_bam = "$(assembled_fasta).bwa.bam"
    # sorting and deduping
    # if !isfile(mapped_reads_bam)
    #     run(pipeline(
    #         `bwa mem -R "@RG\tID:$(SRR)\tSM:bar" -t $(n_cores) $(assembled_fasta) $(trimmed_forward_reads) $(trimmed_reverse_reads)`,
    #         `samtools collate --threads $(n_cores) -O - -`,
    #         `samtools fixmate --threads $(n_cores) -m - -`,
    #         `samtools sort --threads $(n_cores)`,
    #         `samtools markdup --threads $(n_cores) - -`,
    #         `samtools view --threads $(n_cores) -buh`,
    #         mapped_reads_bams))
    # end
    # sorting - 50Gb - 1594.823 sec
    if !isfile(mapped_reads_bam) || (filesize(mapped_reads_bam) == 0)
        run(pipeline(
            `bwa mem -R "@RG\tID:$(SRR)\tSM:bar" -t $(n_cores) $(assembled_fasta) $(trimmed_forward_reads) $(trimmed_reverse_reads)`,
            `samtools sort --threads $(n_cores) -u -m 100M`,
            `samtools view --threads $(n_cores) -bh`,
            mapped_reads_bam))
    end

    # if !isfile("$(mapped_reads_bam).bai")
    #     run(`samtools index $(mapped_reads_bam)`)
    # end

    qualimap_report_pdf = "$(out_dir)/qualimap/report.pdf"
    qualimap_report_txt = "$(out_dir)/qualimap/genome_results.txt"

    if !isfile(qualimap_report_pdf) || !isfile(qualimap_report_txt)
        run(`
            conda run --live-stream -n viral-pangenome-discovery
            qualimap bamqc
            -nt $(n_cores)
            -bam $(mapped_reads_bam)
            -outdir $(out_dir)/qualimap
            -outformat PDF:HTML
            --output-genome-coverage $(mapped_reads_bam).genome_coverage.txt
            --java-mem-size=4G
            `)
    end
end

# qualimap_contig_coverage_table = Mycelia.parse_qualimap_contig_coverage(qualimap_report_txt)
# qualimap_contig_coverage_table[!, "% Mapped bases"] = round.(qualimap_contig_coverage_table[!, "Mapped bases"] ./ sum(qualimap_contig_coverage_table[!, "Mapped bases"]) .* 100, digits=3);

Progress:  26%|██████████▌                              |  ETA: 0:10:09[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1641134 sequences (240000216 bp)...
[M::process] read 1619484 sequences (240000198 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (925, 67146, 8, 932)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (83, 152, 220)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 494)
[M::mem_pestat] mean and std.dev: (160.20, 100.40)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 631)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (173, 211, 242)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (35, 380)
[M::mem_pestat] mean and std.dev: (207.76, 56.00)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 449)
[M::mem_pestat] skip orientation 

Java memory size is set to 4G
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 37
Max memory (Mb): 4294
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 22532
Chunk of reads size: 1000
Number of threads: 24
Processed 2253 out of 22532 windows...
Processed 4506 out of 22532 windows...
Processed 6759 out of 22532 windows...
Processed 9012 out of 22532 windows...
Processed 11265 out of 22532 windows...
Processed 13518 out of 22532 windows...
Processed 15771 out of 22532 windows...
Processed 18024 out of 22532 windows...
Processed 20277 out of 22532 windows...
Processed 22530 out of 22532 windows...
Total processed windows:22532
Number of reads: 113549328
Number of valid reads: 115430954
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 112406336
Num mapped first of pair: 56540286
Num mapped second of pair: 5586605

2023-03-15 18:20:09 - MEGAHIT v1.2.9
2023-03-15 18:20:09 - Using megahit_core with POPCNT and BMI2 support
2023-03-15 18:20:09 - Convert reads to binary library
2023-03-15 18:22:45 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399545/trim_galore/SRR6399545_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399545/trim_galore/SRR6399545_2_val_2.fq.gz): pe, 72993842 reads, 151 max length'
2023-03-15 18:22:52 - b'INFO  utils/utils.h                 :  152 - Real: 162.5991\tuser: 73.7719\tsys: 7.0524\tmaxrss: 254088'
2023-03-15 18:22:52 - k-max reset to: 141 
2023-03-15 18:22:52 - Start assembly. Number of CPU threads 24 
2023-03-15 18:22:52 - k list: 21,29,39,59,79,99,119,141 
2023-03-15 18:22:52 - Memory used: 121546591027
2023-03-15 18:22:52 - Extract solid (k+1)-mers for k = 21 
2023-03-15 18:27:18 - Build graph for k = 21 
2023-03-15 18:27:58 - Assemble contigs from

Java memory size is set to 4G
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 37
Max memory (Mb): 4294
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 3696
Chunk of reads size: 1000
Number of threads: 24
Processed 369 out of 3696 windows...
Processed 738 out of 3696 windows...
Processed 1107 out of 3696 windows...
Processed 1476 out of 3696 windows...
Processed 1845 out of 3696 windows...
Processed 2214 out of 3696 windows...
Processed 2583 out of 3696 windows...
Processed 2952 out of 3696 windows...
Processed 3321 out of 3696 windows...
Processed 3690 out of 3696 windows...
Total processed windows:3696
Number of reads: 72993842
Number of valid reads: 73906963
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 71751000
Num mapped first of pair: 36146142
Num mapped second of pair: 35604858
Num singletons: 70587

2023-03-15 19:24:52 - MEGAHIT v1.2.9
2023-03-15 19:24:52 - Using megahit_core with POPCNT and BMI2 support
2023-03-15 19:24:52 - Convert reads to binary library
2023-03-15 19:29:53 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399546/trim_galore/SRR6399546_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399546/trim_galore/SRR6399546_2_val_2.fq.gz): pe, 75537484 reads, 151 max length'
2023-03-15 19:29:55 - b'INFO  utils/utils.h                 :  152 - Real: 302.2714\tuser: 82.8614\tsys: 8.1159\tmaxrss: 254008'
2023-03-15 19:29:55 - k-max reset to: 141 
2023-03-15 19:29:55 - Start assembly. Number of CPU threads 24 
2023-03-15 19:29:55 - k list: 21,29,39,59,79,99,119,141 
2023-03-15 19:29:55 - Memory used: 121546591027
2023-03-15 19:29:55 - Extract solid (k+1)-mers for k = 21 
2023-03-15 19:37:05 - Build graph for k = 21 
2023-03-15 19:38:44 - Assemble contigs from

...

Progress:  30%|████████████▎                            |  ETA: 2 days, 2:54:212023-03-16 14:38:24 - MEGAHIT v1.2.9
2023-03-16 14:38:24 - Using megahit_core with POPCNT and BMI2 support
2023-03-16 14:38:24 - Convert reads to binary library
2023-03-16 14:41:25 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399558/trim_galore/SRR6399558_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399558/trim_galore/SRR6399558_2_val_2.fq.gz): pe, 87880368 reads, 151 max length'
2023-03-16 14:41:29 - b'INFO  utils/utils.h                 :  152 - Real: 185.1103\tuser: 88.6659\tsys: 8.4975\tmaxrss: 255216'
2023-03-16 14:41:29 - k-max reset to: 141 
2023-03-16 14:41:29 - Start assembly. Number of CPU threads 24 
2023-03-16 14:41:29 - k list: 21,29,39,59,79,99,119,141 
2023-03-16 14:41:29 - Memory used: 121546591027
2023-03-16 14:41:29 - Extract solid (k+1)-mers for k = 21 
2023-03-16

Java memory size is set to 4G
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 37
Max memory (Mb): 4294
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 6025
Chunk of reads size: 1000
Number of threads: 24
Processed 602 out of 6025 windows...
Processed 1204 out of 6025 windows...
Processed 1806 out of 6025 windows...
Processed 2408 out of 6025 windows...
Processed 3010 out of 6025 windows...
Processed 3612 out of 6025 windows...
Processed 4214 out of 6025 windows...
Processed 4816 out of 6025 windows...
Processed 5418 out of 6025 windows...
Processed 6020 out of 6025 windows...
Total processed windows:6025
Number of reads: 87880368
Number of valid reads: 89534607
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 87005023
Num mapped first of pair: 43867275
Num mapped second of pair: 43137748
Num singletons: 7919

Progress:  30%|████████████▎                            |  ETA: 2 days, 7:02:232023-03-16 16:43:31 - MEGAHIT v1.2.9
2023-03-16 16:43:31 - Using megahit_core with POPCNT and BMI2 support
2023-03-16 16:43:31 - Convert reads to binary library
2023-03-16 16:53:15 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399559/trim_galore/SRR6399559_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399559/trim_galore/SRR6399559_2_val_2.fq.gz): pe, 147666958 reads, 151 max length'
2023-03-16 16:53:18 - b'INFO  utils/utils.h                 :  152 - Real: 587.2864\tuser: 171.9261\tsys: 15.4944\tmaxrss: 256348'
2023-03-16 16:53:18 - k-max reset to: 141 
2023-03-16 16:53:18 - Start assembly. Number of CPU threads 24 
2023-03-16 16:53:18 - k list: 21,29,39,59,79,99,119,141 
2023-03-16 16:53:18 - Memory used: 121546591027
2023-03-16 16:53:18 - Extract solid (k+1)-mers for k = 21 
2023-03

Java memory size is set to 4G
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 37
Max memory (Mb): 4294
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 14144
Chunk of reads size: 1000
Number of threads: 24
Processed 1414 out of 14144 windows...
Processed 2828 out of 14144 windows...
Processed 4242 out of 14144 windows...
Processed 5656 out of 14144 windows...
Processed 7070 out of 14144 windows...
Processed 8484 out of 14144 windows...
Processed 9898 out of 14144 windows...
Processed 11312 out of 14144 windows...
Processed 12726 out of 14144 windows...
Processed 14140 out of 14144 windows...
Total processed windows:14144
Number of reads: 147666958
Number of valid reads: 153985466
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 142183388
Num mapped first of pair: 71240674
Num mapped second of pair: 70942714
N

Progress:  30%|████████████▍                            |  ETA: 2 days, 10:58:502023-03-16 18:46:48 - MEGAHIT v1.2.9
2023-03-16 18:46:48 - Using megahit_core with POPCNT and BMI2 support
2023-03-16 18:46:48 - Convert reads to binary library
2023-03-16 18:52:38 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399560/trim_galore/SRR6399560_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399560/trim_galore/SRR6399560_2_val_2.fq.gz): pe, 150067290 reads, 151 max length'
2023-03-16 18:52:40 - b'INFO  utils/utils.h                 :  152 - Real: 351.8870\tuser: 170.7811\tsys: 16.3976\tmaxrss: 254216'
2023-03-16 18:52:40 - k-max reset to: 141 
2023-03-16 18:52:40 - Start assembly. Number of CPU threads 24 
2023-03-16 18:52:40 - k list: 21,29,39,59,79,99,119,141 
2023-03-16 18:52:40 - Memory used: 121546591027
2023-03-16 18:52:40 - Extract solid (k+1)-mers for k = 21 
2023-0

In [ ]:
# consider deleting the bams after we get the qualimap reports - we don't need them anymore